데이터 불러오기

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# 1. 결측치만
#df1 = pd.read_csv("/content/gdrive/MyDrive/내배캠_날씨_데이터/train_df1.csv")
#val1 = pd.read_csv("/content/gdrive/MyDrive/내배캠_날씨_데이터/val_df1.csv")
# 2. 이상치 제거
#df = pd.read_csv("/content/gdrive/MyDrive/내배캠_날씨_데이터/train_df3.csv")
#val2 = pd.read_csv("/content/gdrive/MyDrive/내배캠_날씨_데이터/val_df1.csv")
# 3. 이상치 대체
df = pd.read_csv("/content/gdrive/MyDrive/내배캠_날씨_데이터/test_heat_결측치대체.csv")
val = pd.read_csv("/content/gdrive/MyDrive/내배캠_날씨_데이터/val_df3.csv")
# 4. 결측치 제거도 안한 버전
#df = pd.read_csv("/content/gdrive/MyDrive/내배캠_날씨_데이터/train_df4.csv")
#val = pd.read_csv("/content/gdrive/MyDrive/내배캠_날씨_데이터/val_df4.csv")
test = pd.read_csv("/content/gdrive/MyDrive/내배캠_날씨_데이터/test_heat.csv")

In [4]:
column_dict = {
    'TM' : '시간',
    'branch_ID' : '지사명',
    'TA' : '기온',
    'WD' : '풍향',
    'WS' : '픙속',
    'RN_DAY' : '일강수량',
    'RN_HR1' : '시간 강수량',
    'HM' : '상대 습도',
    'SI' : '일사량',
    'ta_chi' : '체감온도',
    'heat_demand' : '열수요'
}
test.rename(columns=column_dict, inplace=True)



---



BI-LSTM
탠서플로

In [5]:
pip install tensorflow

In [6]:
pip install scikit-learn

---------

In [7]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, LSTM, Bidirectional, Dense, Dropout, Attention, Concatenate, LayerNormalization, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# 하이퍼파라미터 설정
TIMESTEPS = 124
LSTM_UNITS = 64
DROPOUT_RATE = 0.3
LEARNING_RATE = 0.001
EPOCHS = 50
BATCH_SIZE = 32
PATIENCE = 5
BASE_FEATURES = ['기온', '풍향', '풍속', '일강수량', '시간 강수량', '상대 습도', '일사량', '체감온도', '시각', '요일1']
TARGET = '열수요'

# 시퀀스 생성 함수
def create_sequences(X, y=None, timesteps=TIMESTEPS):
    Xs, ys = [], []
    for i in range(timesteps, len(X)):
        Xs.append(X[i - timesteps:i])
        if y is not None:
            ys.append(y[i])
    return np.array(Xs), (np.array(ys) if y is not None else None)

# 전처리 함수 (누출 방지 + 타겟 스케일링)
def preprocess_data(df, test):
    df_train_enc = pd.get_dummies(df, columns=['지사명'])
    df_val_enc = pd.get_dummies(test, columns=['지사명'])
    df_val_enc = df_val_enc.reindex(columns=df_train_enc.columns, fill_value=0)

    features = [col for col in df_train_enc.columns if col in BASE_FEATURES or col.startswith('지사명_')]

    X_train, X_val = df_train_enc[features], df_val_enc[features]
    y_train, y_val = df_train_enc[TARGET].values, df_val_enc[TARGET].values

    # 피처 스케일러 (훈련 데이터 기준)
    x_scaler = MinMaxScaler()
    X_train_scaled = x_scaler.fit_transform(X_train)
    X_val_scaled = x_scaler.transform(X_val)

    # 타겟 스케일러 (훈련 데이터 기준)
    y_scaler = MinMaxScaler()
    y_train_scaled = y_scaler.fit_transform(y_train.reshape(-1, 1)).flatten()
    y_val_scaled = y_scaler.transform(y_val.reshape(-1, 1)).flatten()

    return X_train_scaled, X_val_scaled, y_train_scaled, y_val_scaled, x_scaler, y_scaler, df_val_enc

# CNN + BiLSTM + Attention 모델 구성
def build_model(input_shape):
    inputs = Input(shape=input_shape)

    x = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(inputs)
    x = MaxPooling1D(pool_size=2)(x)
    x = BatchNormalization()(x)

    x = Bidirectional(LSTM(LSTM_UNITS, return_sequences=True))(x)
    x = Dropout(DROPOUT_RATE)(x)
    x = Bidirectional(LSTM(LSTM_UNITS // 2, return_sequences=True))(x)
    x = Dropout(DROPOUT_RATE * 0.5)(x)

    attention = Attention()([x, x])
    x = Concatenate()([x, attention])

    x = Dense(64, activation='relu')(x)
    x = LayerNormalization()(x)
    x = Dense(32, activation='relu')(x)
    x = Dense(1)(x)

    x = x[:, -1, :]  # 시퀀스 중 마지막 시점 값 추출
    model = Model(inputs, x)
    model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss='mse', metrics=['mae'])
    return model

# 학습 및 평가

def run_model(df, test):
    X_train_scaled, X_val_scaled, y_train_scaled, y_val_scaled, x_scaler, y_scaler, val_processed = preprocess_data(df, test)

    X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled)
    X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled)

    model = build_model((TIMESTEPS, X_train_seq.shape[2]))
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=PATIENCE, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
    ]

    model.fit(
        X_train_seq, y_train_seq,
        validation_data=(X_val_seq, y_val_seq),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=callbacks,
        verbose=1
    )
    return model, x_scaler, y_scaler, val_processed, (X_val_seq, y_val_seq)

# 평가 함수
def evaluate_model(model, X_val_seq, y_val_seq, y_scaler):
    y_pred_scaled = model.predict(X_val_seq).flatten()
    y_pred = y_scaler.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()
    y_true = y_scaler.inverse_transform(y_val_seq.reshape(-1, 1)).flatten()

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)

    print(f"📊 MAE: {mae:.2f} | RMSE: {rmse:.2f} | R²: {r2:.4f}")
    return y_pred

# 테스트 예측 함수
def predict_on_test(model, test_df_raw, x_scaler, y_scaler, reference_df):
    test_enc = pd.get_dummies(test_df_raw, columns=['지사명'])
    test_enc = test_enc.reindex(columns=reference_df.columns, fill_value=0)
    features = [col for col in reference_df.columns if col in BASE_FEATURES or col.startswith('지사명_')]
    X_test = test_enc[features]
    X_test_scaled = x_scaler.transform(X_test)

    X_test_seq, _ = create_sequences(X_test_scaled)
    y_pred_scaled = model.predict(X_test_seq).flatten()
    y_pred = y_scaler.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()

    result_df = test_df_raw.iloc[TIMESTEPS:].copy()
    result_df['예측_열수요'] = y_pred
    return result_df


In [ ]:
X_train_scaled, X_val_scaled, y_train_scaled, y_val_scaled, x_scaler, y_scaler, val_processed = preprocess_data(df, val)
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled)

# 3. 모델 학습
input_shape = (TIMESTEPS, X_train_seq.shape[2])
model = build_model(input_shape)

callbacks = [
    EarlyStopping(monitor='val_loss', patience=PATIENCE, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
]

model.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_val_seq, y_val_seq),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks,
    verbose=1
)

# 4. 평가
evaluate_model(model, X_val_seq, y_val_seq, y_scaler)

# 5. 테스트 예측
result_df = predict_on_test(model, test, x_scaler, y_scaler, reference_df=val_processed)
result_df.to_csv("CNN_BiLSTM_Attention_예측결과.csv", index=False)
print("✅ 예측 결과 저장 완료 → CNN_BiLSTM_Attention_예측결과.csv")



---



In [ ]:
# 라이브러리 임포트
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# 하이퍼파라미터 설정
TIMESTEPS = 124
LSTM_UNITS = 64
DROPOUT_RATE = 0.3
LEARNING_RATE = 0.01
BATCH_SIZE = 32
EPOCHS = 50
PATIENCE = 5


# 특성 및 타깃 설정
BASE_FEATURES = ['기온', '풍향', '풍속', '일강수량', '시간 강수량',
                 '상대 습도', '일사량', '체감온도', '시각', '요일1']
TARGET = '열수요'

def create_sequences(X, y=None, timesteps=TIMESTEPS):
    Xs, ys = [], []
    for i in range(timesteps, len(X)):
        Xs.append(X[i - timesteps:i])
        if y is not None:
            ys.append(y[i])
    return np.array(Xs), (np.array(ys) if y is not None else None)

def preprocess_data(df, test):
    df_train_enc = pd.get_dummies(df, columns=['지사명'])
    df_val_enc = pd.get_dummies(test, columns=['지사명'])
    df_val_enc = df_val_enc.reindex(columns=df_train_enc.columns, fill_value=0)

    features = [col for col in df_train_enc.columns if col in BASE_FEATURES or col.startswith('지사명_')]
    X_train, X_val = df_train_enc[features], df_val_enc[features]
    y_train, y_val = df_train_enc[TARGET].values, df_val_enc[TARGET].values

    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    return X_train_scaled, X_val_scaled, y_train, y_val, scaler, df_val_enc

def build_model(input_shape, hidden_layers=[64, 32], hidden_activation='relu'):
    model = Sequential()

    # 1. LSTM Block
    model.add(Bidirectional(LSTM(LSTM_UNITS, return_sequences=True), input_shape=input_shape))
    model.add(Dropout(DROPOUT_RATE))
    model.add(Bidirectional(LSTM(LSTM_UNITS // 2, return_sequences=False)))
    model.add(Dropout(DROPOUT_RATE * 0.5))

    # 2. Dense Hidden Layers (유연하게 구성)
    for units in hidden_layers:
        model.add(Dense(units, activation=hidden_activation))

    # 3. Output Layer
    model.add(Dense(1))

    model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss='mse', metrics=['mae'])
    return model


def run_model(df, test):
    print("🔄 데이터 전처리...")
    X_train_scaled, X_val_scaled, y_train, y_val, scaler, val_processed = preprocess_data(df, test)

    print("🔄 시퀀스 생성...")
    X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train)
    X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val)

    print("🏗️ 모델 구축...")
    model = build_model((TIMESTEPS, X_train_seq.shape[2]))

    print("🚀 모델 학습...")
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=PATIENCE, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
    ]

    history = model.fit(
        X_train_seq, y_train_seq,
        validation_data=(X_val_seq, y_val_seq),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=callbacks,
        verbose=1
    )

    return model, history, scaler, (X_val_scaled, y_val), val_processed, (X_val_seq, y_val_seq)

def evaluate_model(model, X_val_seq, y_val_seq):
    print("📊 모델 평가 중...")
    y_pred = model.predict(X_val_seq).flatten()
    mae = mean_absolute_error(y_val_seq, y_pred)
    rmse = np.sqrt(mean_squared_error(y_val_seq, y_pred))
    r2 = r2_score(y_val_seq, y_pred)

    print(f"📈 MAE:  {mae:.2f}")
    print(f"📉 RMSE: {rmse:.2f}")
    print(f"🧮 R²:   {r2:.4f}")

    return y_pred, mae, rmse, r2

def predict_on_test(model, df_test_raw, scaler, reference_df):
    df_test_enc = pd.get_dummies(df_test_raw, columns=['지사명'])
    df_test_enc = df_test_enc.reindex(columns=reference_df.columns, fill_value=0)

    features = [col for col in reference_df.columns if col in BASE_FEATURES or col.startswith('지사명_')]
    X_test = df_test_enc[features]
    X_test_scaled = scaler.transform(X_test)

    X_test_seq, _ = create_sequences(X_test_scaled, timesteps=TIMESTEPS)
    y_pred = model.predict(X_test_seq).flatten()

    df_result = df_test_raw.iloc[TIMESTEPS:].copy()
    df_result['예측_열수요'] = y_pred
    return df_result

# 2. train/validation 분리
train, val = train_test_split(df, test_size=0.2, shuffle=False)

# 3. 학습
model, history, scaler, _, val_processed, (X_val_seq, y_val_seq) = run_model(train, val)

# 4. 평가
evaluate_model(model, X_val_seq, y_val_seq)

# 5. 예측
result_df = predict_on_test(model, test, scaler, reference_df=val_processed)

🔄 데이터 전처리...
🔄 시퀀스 생성...
🏗️ 모델 구축...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


🚀 모델 학습...
